# Deep Learning RNN Model for Used Motorcycle Price Prediction
## IS794 AL - Deep Learning (Lab)
Kelompok B: <br>
- Daffa Kaisha Pratama Chandra (00000055770)
- Justine Donovan (00000063202)
- Marsella Patricia (00000063226)
- Reina Ratu Balqis (00000065099)
---
### Import Libraries

In [1]:
# Basic library
import pandas as pd
import numpy as np
from scipy import stats

# Pre-processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Deep learning
import tensorflow as tf
from tensorflow import keras

# Ignore warning
import warnings
warnings.simplefilter("ignore")

---
Baca Dataset

In [2]:
yamaha = pd.read_csv("motor_yamaha_bekas.csv", index_col=False)
honda = pd.read_csv("motor_honda_bekas.csv", index_col=False)
kawasaki = pd.read_csv("motor_kawasaki_bekas.csv", index_col=False)
suzuki = pd.read_csv("motor_suzuki_bekas.csv", index_col=False)

Penjelasan singkat mengenai dataset yang digunakan: <br>
Dataset ini merupakan hasil web scraping dari website https://momotor.id/

In [3]:
print("Shape dari tiap dataset:")
print(f"Dataset Yamaha memiliki {yamaha.shape[1]} kolom dan {yamaha.shape[0]} baris.")
print(f"Dataset Honda memiliki {honda.shape[1]} kolom dan {honda.shape[0]} baris.")
print(f"Dataset Kawasaki memiliki {kawasaki.shape[1]} kolom dan {kawasaki.shape[0]} baris.")
print(f"Dataset Suzuki memiliki {suzuki.shape[1]} kolom dan {suzuki.shape[0]} baris.")

Shape dari tiap dataset:
Dataset Yamaha memiliki 10 kolom dan 588 baris.
Dataset Honda memiliki 10 kolom dan 1008 baris.
Dataset Kawasaki memiliki 10 kolom dan 348 baris.
Dataset Suzuki memiliki 10 kolom dan 348 baris.


In [4]:
print("Informasi singkat dari tiap dataset:\n")
print("Yamaha:")
print(yamaha.info())
print("\nHonda:")
print(honda.info())
print("\nKawasaki:")
print(kawasaki.info())
print("\nSuzuki:")
print(suzuki.info())

Informasi singkat dari tiap dataset:

Yamaha:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588 entries, 0 to 587
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   nama_motor       343 non-null    object 
 1   lokasi           343 non-null    object 
 2   harga            343 non-null    object 
 3   deskripsi        343 non-null    object 
 4   kapasitas_mesin  343 non-null    object 
 5   tahun            343 non-null    float64
 6   transmisi        343 non-null    object 
 7   kilometer        343 non-null    object 
 8   url              588 non-null    object 
 9   merk             588 non-null    object 
dtypes: float64(1), object(9)
memory usage: 46.1+ KB
None

Honda:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   nama_motor       756 non-nu

Pre-process I: Data mentah -> Data siap pakai
Akan dilakukan cek <i> missing values. </i> Jika terdapat <i> missing values, </i> data yang hilang akan diimputasi menggunakan nilai modus. 

In [5]:
# Cek rata-rata missing values pada tiap dataset
brand = ['yamaha', 'honda', 'kawasaki', 'suzuki']
data = [yamaha, honda, kawasaki, suzuki]

for i, j in zip(brand, data):
    print(f"{i} missing values: \n{np.round(honda.isnull().mean(), 2)*100}")
    print()

yamaha missing values: 
nama_motor         25.0
lokasi             25.0
harga              25.0
deskripsi          25.0
kapasitas_mesin    25.0
tahun              25.0
transmisi          25.0
kilometer          25.0
url                 0.0
merk                0.0
dtype: float64

honda missing values: 
nama_motor         25.0
lokasi             25.0
harga              25.0
deskripsi          25.0
kapasitas_mesin    25.0
tahun              25.0
transmisi          25.0
kilometer          25.0
url                 0.0
merk                0.0
dtype: float64

kawasaki missing values: 
nama_motor         25.0
lokasi             25.0
harga              25.0
deskripsi          25.0
kapasitas_mesin    25.0
tahun              25.0
transmisi          25.0
kilometer          25.0
url                 0.0
merk                0.0
dtype: float64

suzuki missing values: 
nama_motor         25.0
lokasi             25.0
harga              25.0
deskripsi          25.0
kapasitas_mesin    25.0
tahun          

In [6]:
# cari modus dari tiap kolom yang terdapat missing values
kolom_null = ['nama_motor', 'lokasi', 'harga', 'deskripsi', 
              'kapasitas_mesin', 'tahun', 'transmisi', 'kilometer']

In [7]:
# Cari modus dan replace NaN dengan modus
for col in kolom_null:
    # Yamaha
    modus_yamaha = yamaha[col].mode()[0]
    yamaha[col] = yamaha[col].fillna(modus_yamaha)
    
    #Honda
    modus_honda = honda[col].mode()[0]
    honda[col] = honda[col].fillna(modus_honda)
    
    #Kawasaki
    modus_kawasaki = kawasaki[col].mode()[0]
    kawasaki[col] = kawasaki[col].fillna(modus_kawasaki)
    
    #Honda
    modus_suzuki = suzuki[col].mode()[0]
    suzuki[col] = suzuki[col].fillna(modus_suzuki)

In [8]:
# Cek missing values lagi
for i, j in zip(brand, data):
    print(f"{i} missing values: \n{np.round(honda.isnull().mean(), 2)*100}")
    print()

yamaha missing values: 
nama_motor         0.0
lokasi             0.0
harga              0.0
deskripsi          0.0
kapasitas_mesin    0.0
tahun              0.0
transmisi          0.0
kilometer          0.0
url                0.0
merk               0.0
dtype: float64

honda missing values: 
nama_motor         0.0
lokasi             0.0
harga              0.0
deskripsi          0.0
kapasitas_mesin    0.0
tahun              0.0
transmisi          0.0
kilometer          0.0
url                0.0
merk               0.0
dtype: float64

kawasaki missing values: 
nama_motor         0.0
lokasi             0.0
harga              0.0
deskripsi          0.0
kapasitas_mesin    0.0
tahun              0.0
transmisi          0.0
kilometer          0.0
url                0.0
merk               0.0
dtype: float64

suzuki missing values: 
nama_motor         0.0
lokasi             0.0
harga              0.0
deskripsi          0.0
kapasitas_mesin    0.0
tahun              0.0
transmisi          0.0
kilo

In [9]:
# Merge dataset
used_motor = pd.concat([yamaha, honda, kawasaki, suzuki])
print("Shape dari dataset yang digabung: ", used_motor.shape)
used_motor.head()

Shape dari dataset yang digabung:  (2292, 10)


,nama_motor,lokasi,harga,deskripsi,kapasitas_mesin,tahun,transmisi,kilometer,url,merk
0,Yamaha All New Nmax 155,Kota Medan,"Rp17,500,000","Ss lengkap , plat pajak hidup , nego di tempat",125 CC,2019.0,Otomatis,>30000 KM,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
1,Yamaha All New Nmax 155,Kota Medan,"Rp17,500,000","Ss lengkap , plat pajak hidup , nego di tempat",125 CC,2019.0,Otomatis,>30000 KM,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
2,Yamaha All New Nmax 155,Kota Medan,"Rp17,500,000","Ss lengkap , plat pajak hidup , nego di tempat",125 CC,2019.0,Otomatis,>30000 KM,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
3,Yamaha All New Nmax 155,Kota Medan,"Rp17,500,000","Ss lengkap , plat pajak hidup , nego di tempat",125 CC,2019.0,Otomatis,>30000 KM,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha
4,Yamaha All New Nmax 155,Kota Medan,"Rp17,500,000","Ss lengkap , plat pajak hidup , nego di tempat",125 CC,2019.0,Otomatis,>30000 KM,https://www.momotor.id/motor-bekas/yamaha/yama...,Yamaha


---
#### Exploratory Data Analysis <br>
Pada tahap EDA, akan dilakukan ....

---
#### Data Pre-processing
Pada tahap ini, akan dilakukan:
- <i>remove missing values</i>
- 

---
#### Modeling
Pada tahap ....

Tuning Hyperparameter <br>
....

Finalisasi Model
....

---
#### Evaluasi

Terlihat model .....

---
#### Save model
Model yang dengan akurasi tertinggi akan disimpan untuk di <i> deploy </i>